# Prediction

Deployment of kserve will take few minutes to comes to ready state. In just a short span of time, it will transition
into a ready state and you'll be in a state where you can invoke the service you deployed. You can check the status of
your deployment by navigating to the Kubeflow Endopoints UI.

In [ ]:
import os
import requests
import ipywidgets as widgets

from IPython.display import display

In [ ]:
# Add heading
heading = widgets.HTML("<h2>Credentials</h2>")
display(heading)

domain_input = widgets.Text(description='Username:', placeholder="i001ua.tryezmeral.com")
username_input = widgets.Text(description='Username:')
password_input = widgets.Password(description='Password:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

domain = None
username = None
password = None

def submit_button_clicked(b):
    global domain, username, password
    domain = domain_input.value
    username = username_input.value
    password = password_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(domain_input, username_input, password_input, submit_button, success_message)

In [ ]:
token_url = f"https://keycloak.{domain}/realms/UA/protocol/openid-connect/token"

data = {
    "username" : username,
    "password" : password,
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

In [ ]:
DOMAIN_NAME = "svc.cluster.local"  # change this to your domain for external access
NAMESPACE = open("/var/run/secrets/kubernetes.io/serviceaccount/namespace", "r").read()
DEPLOYMENT_NAME = "bike-sharing"
MODEL_NAME = DEPLOYMENT_NAME
SVC = f"{DEPLOYMENT_NAME}-predictor.{NAMESPACE}.{DOMAIN_NAME}"
URL = f"https://{SVC}/v2/models/{MODEL_NAME}/infer"

print(URL)

In [ ]:
names = ["season", "year", "month", "hour_of_day", "is_holiday", "weekday",
         "is_workingday", "weather_situation", "temperature",
         "feels_like_temperature", "humidity", "windspeed"]

input_data = [
    [1, 2, 1, 0, 0, 6, 0, 1, 0.24, 0.2879, 0.81, 0.0000],
    [1, 5, 1, 0, 0, 6, 1, 1, 0.24, 0.2879, 0.81, 0.0000]
]

inputs = {
  "inputs": [
    {
      "name": "ndarray",
      "shape": [2, 12],
      "datatype": "FP32",
      "data": input_data
    }
  ]
}

headers = {"Authorization": f"Bearer {token}"}
response = requests.post(URL, json=inputs, headers=headers, verify=False)

print(f"Rented Bikes Per Hour: {response.json()['outputs'][0]['data']}")